# Оптимизация графа вычислений на примере слияния Conv–BatchNorm–ReLU

## 1. Введение

Во время оптимизации графа вычислений фреймворки, такие как ONNX Runtime, TensorRT или OpenVINO, объединяют последовательные операции свёртки (Conv), нормализации (BatchNorm) и активации (ReLU) в одну эквивалентную, но более эффективную операцию.  

Основная причина этого — уменьшение количества обращений к памяти и запусков отдельных вычислительных ядер на GPU или CPU.  
Главное математическое основание: и свёртка, и нормализация представляют собой **линейные преобразования**, которые могут быть объединены в одно эквивалентное линейное преобразование.

## 2. Свёрточная операция (Convolution)

Свёртка в свёрточной нейронной сети — это операция линейного преобразования, вычисляемая по формуле

$$
Y[i,j] = \sum_{u=0}^{H_W-1} \sum_{v=0}^{W_W-1} W[u,v] \cdot X[i+u, j+v] + b
$$

где  
-  X  — входное изображение,  
-  W  — фильтр (ядро свёртки),  
-  b  — смещение (bias),  
-  Y  — выход свёртки,  
-  H_W, W_W  — высота и ширина фильтра.

### 2.1. Параметры свёртки

- **Stride (шаг свёртки)** — насколько ядро сдвигается по входу.  
- **Padding** — добавление рамки нулей вокруг входа.  
- **Valid padding** — свёртка без добавления нулей (padding = 0).  
- **Same padding** — паддинг подбирается так, чтобы выход имел те же размеры, что и вход.

Для valid-свёртки (padding = 0) размер выхода равен:

$$
H_{out} = H_{in} - H_W + 1, \quad
W_{out} = W_{in} - W_W + 1
$$

## 3. Batch Normalization

Batch Normalization в режиме обучения нормализует активации внутри батча, а затем масштабирует и сдвигает их с помощью обучаемых параметров $ \gamma $ и $ \beta $:

$$
\text{BN}(y) = \gamma \frac{y - \mu_{\text{batch}}}{\sqrt{\sigma^2_{\text{batch}} + \epsilon}} + \beta
$$

- $ \mu_{\text{batch}} $ — среднее по батчу,  
- $ \sigma^2_{\text{batch}} $ — дисперсия по батчу,  
- $ \epsilon $ — стабилизирующая константа,  
- $ \gamma, \beta $ — обучаемые параметры.

### 3.1. BatchNorm на inference

На этапе inference используются накопленные статистики (running mean и running variance):

$$
\text{BN}(y) = \gamma \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

Это линейная операция над каждым элементом $ y $, которую можно записать в виде

$$
\text{BN}(y) = a y + c,
$$

где

$$
a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}, \quad c = \beta - a \mu.
$$

#### Преобразование формулы Batch Normalization

Рассмотрим пошаговое преобразование формулы Batch Normalization в режиме inference, чтобы показать, как она сводится к линейной форме.

---

##### Шаг 1. Определение BatchNorm в режиме inference

В режиме вывода (inference) Batch Normalization вычисляется по формуле:

$$
\text{BN}(y) = \gamma \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

где  
- $\gamma$ и $\beta$ — обучаемые параметры (масштаб и смещение),  
- $\mu$ и $\sigma^2$ — накопленные оценки среднего и дисперсии,  
- $\epsilon$ — малая добавка для численной стабильности.

---

##### Шаг 2. Вынесение константы перед скобкой

Выражение под дробью можно переписать как:

$$
\frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} =
\frac{1}{\sqrt{\sigma^2 + \epsilon}} \cdot (y - \mu)
$$

Тем самым знаменатель $\sqrt{\sigma^2 + \epsilon}$ выносится за скобку как константа.

---

##### Шаг 3. Введение коэффициента масштабирования

Теперь умножим на параметр $\gamma$:

$$
\gamma \cdot \frac{1}{\sqrt{\sigma^2 + \epsilon}}
= \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} = a
$$

Коэффициент $a$ представляет собой масштаб, с которым нормализованные значения $y$ будут растянуты.  
Например, при $\gamma = 3$ и $\sigma^2 = 4$:

$$
a = \frac{3}{\sqrt{4}} = \frac{3}{2} = 1.5
$$

---

##### Шаг 4. Раскрытие скобок

Подставим определение $a$ в формулу:

$$
\gamma \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} = a(y - \mu)
$$

Раскрывая скобки, получаем:

$$
a(y - \mu) = a y - a \mu
$$

Это простое алгебраическое разложение, показывающее, что нормализация представляет собой линейную комбинацию исходного $y$.

---

##### Шаг 5. Добавление параметра смещения

После раскрытия скобок добавляем $\beta$:

$$
\text{BN}(y) = a(y - \mu) + \beta
$$

Раскроем выражение полностью:

$$
\text{BN}(y) = a y - a \mu + \beta
$$

Сгруппируем слагаемые по смыслу:

- часть $a y$ отвечает за масштабирование,  
- часть $(\beta - a \mu)$ отвечает за смещение.

---

### Итоговая форма

Итоговая формула Batch Normalization в линейной форме:

$$
\text{BN}(y) = a y + (\beta - a \mu)
$$

## 4. Слияние Conv и BatchNorm

### Числовой пример

### Шаг 1. Свёртка $Y = W * X + b$

#### Исходные данные

- Входное изображение $X \in \mathbb{R}^{3 \times 3}$:

$$
X =
\begin{bmatrix}
1 & 2 & 0 \\
0 & 1 & 3 \\
2 & 2 & 1
\end{bmatrix}
$$

- Фильтр (ядро свёртки) $W \in \mathbb{R}^{2 \times 2}$:

$$
W =
\begin{bmatrix}
1 & 0 \\
-1 & 2
\end{bmatrix}
$$

- Смещение (bias): $b = 0$

- Параметры свёртки:
  - шаг (stride) = 1  
  - заполнение (padding) = 0  
  - режим — **valid**, то есть без добавления нулей.

---

#### Размер выхода

Для свёртки с valid-паддингом выходной размер вычисляется по формулам:

$$
H_{out} = H_{in} - H_W + 1, \quad
W_{out} = W_{in} - W_W + 1
$$

Подставляя значения $H_{in} = 3$, $H_W = 2$:

$$
H_{out} = 3 - 2 + 1 = 2, \quad W_{out} = 2
$$

Следовательно, результат свёртки $Y \in \mathbb{R}^{2 \times 2}$.

---

#### Формула элемента выхода

Каждый элемент $Y[i, j]$ вычисляется как:

$$
Y[i,j] = \sum_{u=0}^{1}\sum_{v=0}^{1} W[u,v] \cdot X[i+u, j+v] + b
$$

---

#### Пошаговые расчёты элементов выхода

1. **Элемент $Y[0,0]$**  
   Окно входа $X[0:2, 0:2]$:

   $$
   \begin{bmatrix}
   1 & 2 \\
   0 & 1
   \end{bmatrix}
   $$

   Тогда

   $$
   Y[0,0] = 1 \cdot 1 + 0 \cdot 2 + (-1) \cdot 0 + 2 \cdot 1 + 0 = 3
   $$

2. **Элемент $Y[0,1]$**  
   Окно входа $X[0:2, 1:3]$:

   $$
   \begin{bmatrix}
   2 & 0 \\
   1 & 3
   \end{bmatrix}
   $$

   $$
   Y[0,1] = 1 \cdot 2 + 0 \cdot 0 + (-1) \cdot 1 + 2 \cdot 3 = 7
   $$

3. **Элемент $Y[1,0]$**  
   Окно входа $X[1:3, 0:2]$:

   $$
   \begin{bmatrix}
   0 & 1 \\
   2 & 2
   \end{bmatrix}
   $$

   $$
   Y[1,0] = 1 \cdot 0 + 0 \cdot 1 + (-1) \cdot 2 + 2 \cdot 2 = 2
   $$

4. **Элемент $Y[1,1]$**  
   Окно входа $X[1:3, 1:3]$:

   $$
   \begin{bmatrix}
   1 & 3 \\
   2 & 1
   \end{bmatrix}
   $$

   $$
   Y[1,1] = 1 \cdot 1 + 0 \cdot 3 + (-1) \cdot 2 + 2 \cdot 1 = 1
   $$

---

#### Итог результата свёртки

$$
Y =
\begin{bmatrix}
3 & 7 \\
2 & 1
\end{bmatrix}
$$

### Шаг 2. Batch Normalization на этапе inference

#### Формула Batch Normalization

В режиме inference Batch Normalization вычисляется по формуле:

$$
\text{BN}(y) = \gamma \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

где  
- $\gamma$ — обучаемый параметр масштабирования,  
- $\beta$ — обучаемый параметр смещения,  
- $\mu$ — среднее значение, накопленное за время обучения,  
- $\sigma^2$ — накопленная дисперсия,  
- $\epsilon$ — малая добавка для предотвращения деления на ноль.

Эта формула представляет собой **линейное преобразование** выходных значений свёртки.

---



### Шаг 3. Слияние Batch Normalization со свёрткой

На предыдущем шаге было показано, что Batch Normalization при inference сводится к линейному преобразованию вида:

$$
\text{BN}(y) = a y + (\beta - a \mu),
$$

где  
$a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}$ — коэффициент масштабирования,  
$\beta - a\mu$ — коэффициент смещения.

Теперь подставим в эту формулу результат свёртки $Y = W * X + b$.

---

#### Подстановка выражения свёртки

$$
\text{BN}(W * X + b)
= a(W * X + b) + (\beta - a\mu)
$$

Раскроем скобки:

$$
= a(W * X) + a b + (\beta - a\mu)
$$

Сгруппируем члены, зависящие от входа $X$, и постоянные члены отдельно:

$$
= (a W) * X + (a b + \beta - a\mu)
$$

---

#### Определение новых параметров свёртки

Сравнивая это выражение с формулой стандартной свёртки

$$
Y' = W' * X + b',
$$

можно определить новые параметры:

$$
W' = a W, \quad b' = a b + \beta - a\mu.
$$

Таким образом, последовательное применение операций **Conv → BatchNorm** эквивалентно одной свёртке с новыми параметрами $W'$ и $b'$.

---

#### Интерпретация

- $W'$ — новая матрица фильтров, полученная масштабированием исходных весов на коэффициент $a$.  
- $b'$ — новое смещение, учитывающее как исходный bias, так и влияние параметров нормализации $\gamma$, $\beta$, $\mu$ и $\sigma^2$.

#### Подставляем параметры примера

Используем данные, полученные на предыдущем шаге:

- $\gamma = 3$
- $\beta = -1$
- $\mu = 2$
- $\sigma^2 = 4$
- $\epsilon = 0$

Результат свёртки из Шага 1:

$$
Y =
\begin{bmatrix}
3 & 7 \\
2 & 1
\end{bmatrix}
$$

---

#### Расчёт коэффициентов нормализации

Вычисляем знаменатель:

$$
\sqrt{\sigma^2 + \epsilon} = \sqrt{4} = 2
$$

Коэффициент масштабирования:

$$
a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} = \frac{3}{2} = 1.5
$$

Теперь формула BatchNorm принимает вид:

$$
\text{BN}(y) = 1.5 \cdot y + (\beta - a \mu)
$$

---

#### Расчёт смещения

Подставим известные значения:

$$
\beta - a \mu = -1 - (1.5 \cdot 2) = -1 - 3 = -4
$$

Тогда итоговая формула для BatchNorm:

$$
\text{BN}(y) = 1.5 \cdot y - 4
$$

---

#### Применение BatchNorm покомпонентно

Каждый элемент матрицы $Y$ масштабируется и сдвигается по формуле выше:

$$
\begin{aligned}
1.5 \cdot 3 - 4 &= 0.5 \\
1.5 \cdot 7 - 4 &= 6.5 \\
1.5 \cdot 2 - 4 &= -1.0 \\
1.5 \cdot 1 - 4 &= -2.5
\end{aligned}
$$

Получаем результат:

$$
\text{BN}(Y) =
\begin{bmatrix}
0.5 & 6.5 \\
-1.0 & -2.5
\end{bmatrix}
$$

## 5. Добавление ReLU

Функция активации ReLU определяется как

$$
\text{ReLU}(x) = \max(0, x).
$$

Хотя ReLU не является линейной операцией, она элемент-wise и не изменяет размерность выходного тензора.  
Поэтому backend-системы могут включить её непосредственно в тот же вычислительный kernel, выполняя операцию активации сразу после свёртки без записи промежуточных данных в память.

Результатом становится **fused-kernel** (например, `ConvBnRelu`), выполняющий все три операции за один проход.

## 6. Оптимизация графа в ONNX Runtime и TensorRT

Современные backend-системы выполняют оптимизацию графа вычислений в несколько этапов.

1. **Поиск паттернов**  
   Граф анализируется на наличие последовательностей узлов Conv → BatchNorm → ReLU.

2. **Пересчёт параметров**  
   Для каждой найденной последовательности вычисляются:
   $$
   a = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}, \quad c = \beta - a\mu,
   $$
   $$
   W' = aW, \quad b' = ab + c.
   $$

3. **Переписывание графа**  
   Узлы BatchNorm и ReLU удаляются, а Conv заменяется новой операцией с параметрами $ W', b' $.  
   В некоторых системах (например, TensorRT) создаётся специальный узел `FusedConv`.

4. **Преимущества**  
   - Уменьшается количество обращений к памяти.  
   - Сокращается число запусков GPU-ядер.  
   - Снижается накладной overhead синхронизации.  
   - Повышается общая пропускная способность вычислений.

## 7. Паттерны оптимизации

ONNX и TensorRT поддерживают оптимизацию и объединение различных паттернов нейронных сетей, помимо классического сочетания convolution + batch normalization + ReLU (CBR). Некоторые из них включают:

1. **Свёрточные слои с добавлением смещения (bias) и активацией**.

2. **Полносвязные (линейные) слои с активацией**.

3. **Операции пулинга (Pooling) с последующими слоями**.

4. **Нормализация по группам (Group Normalization)**.

5. **Операции квантования и деквантования (Q/DQ)**.

6. **Трансформер-архитектуры (например, BERT, GPT)**.

7. **Динамические формы входных данных**.

8. **Операции с разреженными матрицами**.

9. **Кастомные операции через плагины**.

10. **Оптимизация для конкретных GPU**.

Эти паттерны и оптимизации позволяют значительно ускорить инференс моделей на NVIDIA GPU, сокращая задержку и повышая пропускную способность.

## 7. Вывод

Слияние Conv, BatchNorm и ReLU основано на линейности их математических преобразований.  
BatchNorm при inference представляет собой линейную операцию, поэтому её можно объединить с параметрами свёртки:

$$
W' = aW, \quad b' = ab + (\beta - a\mu).
$$

Это позволяет сократить вычислительный граф без потери точности результатов.  
В оптимизированном графе фреймворк выполняет эквивалентную последовательность операций значительно быстрее за счёт уменьшения обращений к памяти и объединения вычислений в единый fused kernel.
